# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.243457174417388),
 (1, 1.243457174417388),
 (2, 1.243457174417388),
 (3, 1.243457174417388),
 (4, 1.243457174417388)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('banana', 2)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.020803537592738675),
   (None, 0.04869788955664345),
   (None, 0.10154060019963296),
   (None, 0.13445343446056512),
   (None, 0.15335786105204996),
   (None, 0.16435200467042144),
   (None, 0.2865948595965916),
   (None, 0.3301902849691952),
   (None, 0.3514180314740667),
   (None, 0.35461344277024964),
   (None, 0.45063148699614874),
   (None, 0.4576409491483936),
   (None, 0.4883423389367645)]),
 (1,
  [(None, 0.5544386487103019),
   (None, 0.5753038158657275),
   (None, 0.6182781924026896),
   (None, 0.6425299010180078),
   (None, 0.6615043765008273),
   (None, 0.6701000429120353),
   (None, 0.7215959177686226),
   (None, 0.7617948482247091),
   (None, 0.7682587982199014),
   (None, 0.8050648294917139),
   (None, 0.8067201917352936),
   (None, 0.8318396085507537),
   (None, 0.8354157065233713),
   (None, 0.8589128553606459),
   (None, 0.8888844193086178),
   (None, 0.893806954319194),
   (None, 0.9253111701520373)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

In [26]:
import numpy as np
from typing import NamedTuple
from typing import Iterator

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

input_collection = [
    User(id=0, age=55, gender="male", social_contacts=20),
    User(id=1, age=25, gender="female", social_contacts=240),
    User(id=2, age=25, gender="female", social_contacts=500),
    User(id=3, age=33, gender="female", social_contacts=800),
]

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [27]:
# Одинаковый ключ группировки (0) для всех пользователей
def MAP_MAX(_, row: NamedTuple):
    yield (0, row)

# Поиск пользователя с максимальным числом контактов
def REDUCE_MAX(_, rows: Iterator[NamedTuple]):
    max_contacts = 0
    max_contacts_user = None
    for row in rows:
        if row.social_contacts > max_contacts:
            max_contacts = row.social_contacts
            max_contacts_user = row
    yield max_contacts_user

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

list(MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX))

[User(id=3, age=33, social_contacts=800, gender='female')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [28]:
# Одинаковый ключ группировки (0) для всех пользователей
def MAP_AVG(_, row: NamedTuple):
    yield (0, row)

# Подсчет среднего числа контактов
def REDUCE_AVG(_, rows: Iterator[NamedTuple]):
    sum_contact, i = 0, 0
    for row in rows:
        sum_contact += row.social_contacts
        i += 1
    yield sum_contact / i if i > 0 else 0

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

list(MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG))

[390.0]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [29]:
def groupbykey(iterable):
    # Сортировка по ключу
    iterable = sorted(iterable, key=lambda e: e[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

print(f"max: {list(MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX))}")

print(f"average: {list(MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG))}")

max: [User(id=3, age=33, social_contacts=800, gender='female')]
average: [390.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [30]:
input_collection = [
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=0, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=32, gender="female", social_contacts=753),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    # Разделение массива пользователей между вычислительными узлами
    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

# Группировка по id
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)

# Из пользователей с одинаковым id остается только первый
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)

[(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=54, social_contacts=20, gender='male'),
   User(id=2, age=16, social_contacts=123, gender='male')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [31]:
# Предикат C = проверка параметра gender
def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "female":
        yield (row, row)

def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

list(MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT))

[(User(id=0, age=25, social_contacts=240, gender='female'),
  [User(id=0, age=25, social_contacts=240, gender='female')]),
 (User(id=2, age=35, social_contacts=247, gender='female'),
  [User(id=2, age=35, social_contacts=247, gender='female')]),
 (User(id=3, age=32, social_contacts=753, gender='female'),
  [User(id=3, age=32, social_contacts=753, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [32]:
input_collection_2 = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="undefined", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="undefined", social_contacts=247),
    dict(id=3, age=31, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="female", social_contacts=753),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

# Группировка по id
def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # Удаляем поле gender у полученого словаря
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)

def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]

list(MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION))

[(0,
  [{'id': 0, 'age': 54, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 642}]),
 (2,
  [{'id': 2, 'age': 16, 'gender': 'male', 'social_contacts': 123},
   {'id': 2, 'age': 35, 'social_contacts': 247}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 521},
   {'id': 3, 'age': 32, 'gender': 'female', 'social_contacts': 753}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [33]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=753),
]

# Группировка пользователей по id
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)

# Выходное значение равно (t, t) независимо от присутствия пользователя в только одной или обоих выборках
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

list(MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION))

[(User(id=1, age=54, social_contacts=20, gender='male'),
  User(id=1, age=54, social_contacts=20, gender='male')),
 (User(id=2, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=27, social_contacts=642, gender='male'),
  User(id=3, age=27, social_contacts=642, gender='male')),
 (User(id=4, age=16, social_contacts=123, gender='male'),
  User(id=4, age=16, social_contacts=123, gender='male')),
 (User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')),
 (User(id=6, age=31, social_contacts=521, gender='male'),
  User(id=6, age=31, social_contacts=521, gender='male')),
 (User(id=7, age=32, social_contacts=753, gender='female'),
  User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [34]:
# Группировка по пользователям
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)

# Eсли пользователь есть в обоих выборках
def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

list(MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION))

[[User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [35]:
# Группировка по пользователям
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)

# Пользователи только из первой выборки
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)

# На первом месте стоит номер выборки
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]

list(MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE))

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=240, gender='female'),
 User(id=3, age=27, social_contacts=642, gender='male'),
 User(id=4, age=16, social_contacts=123, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [41]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    city_id: int

users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, city_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, city_id=2),
    User(id=3, age=27, gender="male", social_contacts=642, city_id=2),
    User(id=4, age=16, gender="male", social_contacts=123, city_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, city_id=4),
]

class City(NamedTuple):
    id: int
    name: str

cities_collection = [
    City(id=1, name="Samara"),
    City(id=2, name="Moscow"),
    City(id=3, name="Omsk"),
    City(id=4, name="Tomks"),
    City(id=5, name="Belgorod"),
]

# Группировка пользователей по id города
def MAP_JOIN(city_id, row):
    yield (city_id, row)

def REDUCE_JOIN(city_id, rows):
    users = []
    city = None

    # Определения города
    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            city = row

    for row in rows:
        if type(row) is User:
            # a - пользователь, b - id города, c - сам город
            yield (row, row.city_id, city)

# На первом месте стоит id города, на втором - сам город или пользователь
def RECORDREADER():
    return [(user.city_id, user) for user in users_collection] + [
        (city.id, city) for city in cities_collection
    ]

join_data = list(MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN))
join_data

[(User(id=1, age=54, social_contacts=20, gender='male', city_id=1),
  1,
  City(id=1, name='Samara')),
 (User(id=2, age=25, social_contacts=240, gender='female', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=3, age=27, social_contacts=642, gender='male', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=4, age=16, social_contacts=123, gender='male', city_id=3),
  3,
  City(id=3, name='Omsk')),
 (User(id=5, age=35, social_contacts=247, gender='female', city_id=4),
  4,
  City(id=4, name='Tomks'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [42]:
# Группировка по id города
def MAP_GROUP(city_id, user, city):
    yield (city_id, user)

# Подсчет количества пользователей в каждом из городов
def REDUCE_GROUP(city_id, rows):
    yield f"city with id={city_id} has {len(rows)} user(s)"

def RECORDREADER():
    return [(city_id, user, city) for user, city_id, city in join_data]

list(MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP))

['city with id=1 has 1 user(s)',
 'city with id=2 has 2 user(s)',
 'city with id=3 has 1 user(s)',
 'city with id=4 has 1 user(s)']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [43]:
from typing import List
from typing import Iterator
from typing import Tuple
import numpy as np

ROWS = 5
COLS = 4
STRIPES = 2

COLS_IN_STRIPE = COLS // STRIPES
M = [np.ones((ROWS, COLS_IN_STRIPE)) for i in range(STRIPES)]
v = [np.random.rand(COLS_IN_STRIPE) for i in range(STRIPES)]

def MAP(coordinates: Tuple[int, int], value: int):
  i, j = coordinates
  yield (i, value * vec[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
  print(i, products)
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  global vec
  for stripe_index, stripe in enumerate(M):
    vec = v[stripe_index]
    for i in range(stripe.shape[0]):
      for j in range(stripe.shape[1]):
        yield ((i, j), stripe[i, j])

print(M)
print(v)
list(MapReduce(RECORDREADER, MAP, REDUCE))

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])]
[array([0.48522212, 0.09977812]), array([0.12483688, 0.05603479])]
0 [0.48522211849010544, 0.09977812479796744, 0.1248368836516981, 0.05603478643412818]
1 [0.48522211849010544, 0.09977812479796744, 0.1248368836516981, 0.05603478643412818]
2 [0.48522211849010544, 0.09977812479796744, 0.1248368836516981, 0.05603478643412818]
3 [0.48522211849010544, 0.09977812479796744, 0.1248368836516981, 0.05603478643412818]
4 [0.48522211849010544, 0.09977812479796744, 0.1248368836516981, 0.05603478643412818]


[(0, 0.7658719133738991),
 (1, 0.7658719133738991),
 (2, 0.7658719133738991),
 (3, 0.7658719133738991),
 (4, 0.7658719133738991)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [44]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [45]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)

def REDUCE(key, values):
  (i, k) = key
  k3 = (i, k)

  v3 = 0
  for j in range(J):
      v3 += values[j]
  yield (k3, v3)

Проверьте своё решение

In [46]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [47]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [48]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# Генерирует пары элементов из обоих матриц вместе с соответствующими им индексами
# Элементы в паре предназначены для перемножения и дальнейшего суммирования (M[i,j] и N[i,k])
def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

# Перемножает полученные элементы двух матриц
# Группирует произведения по индексу, соответствующему клетке в результирующей матрице
# Таким образом, для каждого ключа (i, k) генерируется J произведений
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

# Суммирует полученные произведения для вычисления элементов результирующей матрицы
def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [49]:
maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    # Генерируются пары элементов из обоих матриц
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # Разделение между вычислительными узлами по строкам первой матрицы
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

# Возвращает произведения соответствующих элементов из обоих матриц, группируя их по индексу в результирующей матрице
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

# Суммирует полученные произведения
def REDUCE(key, values):
    (i, k) = key

    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

np.allclose(reference_solution, asmatrix(solution))

240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [50]:
from typing import List
from typing import Iterator
import numpy as np

I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)
maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix, shift):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i + shift, j), matrix[i, j])
  split_size = int(np.ceil(len(small_mat) / maps))
  for i in range(0, len(small_mat), split_size):
    yield RECORDREADER(0, small_mat[i:i + split_size], i)
  split_size = int(np.ceil(len(big_mat) / maps))
  for i in range(0, len(big_mat), split_size):
    yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1
  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]
  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]
solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked))

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True